<a href="https://colab.research.google.com/github/YangHong92/ADML-RedHat/blob/colab_test/data_process/svm_takaki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import drive

from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

drive.mount('/content/gdrive') # your google drive will be mounted as '/content/gdrive/My Drive/*'
df_train=pd.read_pickle('/content/gdrive/My Drive/colab/redhat/dataset/train_one_hot_100000.pkl')
df_valid=pd.read_pickle('/content/gdrive/My Drive/colab/redhat/dataset/valid_one_hot_10000.pkl')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
df_train_label=df_train['outcome'].copy()
df_train_input=df_train.drop(columns=['people_id','activity_id','outcome'])

df_valid_label=df_valid['outcome'].copy()
df_valid_input=df_valid.drop(columns=['people_id','activity_id','outcome'])

df_train_input['ppl_char_38']=df_train_input['ppl_char_38']/100
df_valid_input['ppl_char_38']=df_valid_input['ppl_char_38']/100

## Dimension Reduction
To reduce dimension, this code simply removes columns with less number of '1'

In [11]:
# dimension reduction
# the dimensions that only contain small number of '1' are dropped

threshold=0.001 # 0.1%

keys=list(df_train_input)
keys_dropped=[]

for key in keys:
  tmp_sum=df_train_input[key].sum()
  tmp_ratio=tmp_sum/len(df_train_input)
#   print(key,tmp_ratio)
  
  if tmp_ratio < threshold:
    keys_dropped.append(key)
    
df_train_input_reduced=df_train_input.drop(columns=keys_dropped)
df_valid_input_reduced=df_valid_input.drop(columns=keys_dropped)

print('The columns that contain less than ' , threshold*100 ,'% of 1 are removed.')
print(len(keys_dropped),'columns are removed from original',len(keys),'columns.')

The columns that contain less than  0.1 % of 1 are removed.
18253 columns are removed from original 18562 columns.


## Linear SVC

In [12]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

model=LinearSVC(C=0.03125,loss='squared_hinge',penalty='l2',verbose=1)
model.fit(np.array(df_train_input_reduced.values),np.array(df_train_label.values))

train_predict=model.predict(np.array(df_train_input_reduced.values))
train_score=accuracy_score(train_predict,np.array(df_train_label.values))
print('train score is ',train_score)

valid_predict=model.predict(np.array(df_valid_input_reduced.values))
valid_score=accuracy_score(valid_predict,np.array(df_valid_label.values))
print('valid score is ',valid_score)

[LibLinear]train score is  0.86533
valid score is  0.8677


[LibLinear]/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
train score is  0.8663
valid score is  0.8678

## RBF SVM

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

gamma_auto=1.0/(df_train_input_reduced.values.shape[1]*df_train_input_reduced.values.var())
model=SVC(C=10,gamma=gamma_auto*10,verbose=1)
model.fit(np.array(df_train_input_reduced.values),np.array(df_train_label.values))

train_predict=model.predict(np.array(df_train_input_reduced.values))
train_score=accuracy_score(train_predict,np.array(df_train_label.values))
print('train score is ',train_score)

valid_predict=model.predict(np.array(df_valid_input_reduced.values))
valid_score=accuracy_score(valid_predict,np.array(df_valid_label.values))
print('valid score is ',valid_score)

[LibSVM]train score is  0.86801
valid score is  0.8694


[LibSVM]train score is  0.86801
valid score is  0.8694

## Use important 9 features

In [27]:
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score

selected_keys=['ppl_char_38', 'act_date', 'ppl_date', 'ppl_char_6_type 2', 'ppl_group_1_group 17304', 'ppl_char_2_type 2', 'ppl_char_6_type 1', 'ppl_char_6_type 6','ppl_char_7_type 14']

model=LinearSVC(C=0.03125,loss='squared_hinge',penalty='l2',verbose=1)
model.fit(np.array(df_train_input_reduced.loc[:,selected_keys].values),np.array(df_train_label.values))

train_predict=model.predict(np.array(df_train_input_reduced[selected_keys].values))
train_score=accuracy_score(train_predict,np.array(df_train_label.values))
print('train score linear SVM with 9 features is ',train_score)

valid_predict=model.predict(np.array(df_valid_input_reduced[selected_keys].values))
valid_score=accuracy_score(valid_predict,np.array(df_valid_label.values))
print('valid score linear SVM with 9 features is ',valid_score)


gamma_auto=1.0/(df_train_input_reduced.values.shape[1]*df_train_input_reduced.values.var())
model=SVC(C=10,gamma=gamma_auto*10,verbose=1)
model.fit(np.array(df_train_input_reduced.loc[:,selected_keys].values),np.array(df_train_label.values))

train_predict=model.predict(np.array(df_train_input_reduced[selected_keys].values))
train_score=accuracy_score(train_predict,np.array(df_train_label.values))
print('train score RBF SVM with 9 features is ',train_score)

valid_predict=model.predict(np.array(df_valid_input_reduced[selected_keys].values))
valid_score=accuracy_score(valid_predict,np.array(df_valid_label.values))
print('valid score RBF SVM with 9 features is ',valid_score)

[LibLinear]train score linear SVM with 9 features is  0.85965
valid score linear SVM with 9 features is  0.8649
[LibSVM]train score RBF SVM with 9 features is  0.86181
valid score RBF SVM with 9 features is  0.8671


## Use all features

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

model=LinearSVC(C=0.03125,loss='squared_hinge',penalty='l2',verbose=1)
model.fit(np.array(df_train_input.values),np.array(df_train_label.values))

train_predict=model.predict(np.array(df_train_input.values))
train_score=accuracy_score(train_predict,np.array(df_train_label.values))
print('train score is ',train_score)

valid_predict=model.predict(np.array(df_valid_input.values))
valid_score=accuracy_score(valid_predict,np.array(df_valid_label.values))
print('valid score is ',valid_score)

## Source code for Grid Search

In [0]:
# linear svm
model=LinearSVC()
c_cand=np.logspace(-5,15,20,base=2)
parameters = {'penalty':('l1', 'l2'), 'loss': ('hinge','squared_hinge' ),'C':c_cand}

clf=GridSearchCV(model,parameters,cv=3,verbose=0,n_jobs=40,error_score=0.0)
clf.fit(np.array(df_train_input_reduced.values[0:num_sample_to_use]),np.array(df_train_label.values[0:num_sample_to_use]))
print('linear svm result')
print(clf.cv_results_)
print('best params:',clf.best_params_)
print('best score:',clf.best_score_)


gamma_auto=1.0/(df_train_input_reduced.values.shape[1]*df_train_input_reduced.values[0:num_sample_to_use].var())
# gaussian kernel
parameters = {'C':[0.1,0.3,1,3, 10],'gamma':[gamma_auto*0.1,gamma_auto*0.3,gamma_auto*1,gamma_auto*3, gamma_auto*10]}
model=SVC(kernel='rbf',verbose=0)
clf=GridSearchCV(model,parameters,cv=3,verbose=1,n_jobs=64,error_score=0.0)
clf.fit(np.array(df_train_input_reduced.values[0:num_sample_to_use]),np.array(df_train_label.values[0:num_sample_to_use]))


print('rbf svm result')
print(clf.cv_results_)
print('best params:',clf.best_params_)
print('best score:',clf.best_score_)
